# XGBoost

## Run name

In [1]:
import time

project_name = 'XGBoost'
time_str = time.strftime("%Y%m%d-%H%M%S", time.localtime())
run_name = '%s_%s' % (project_name, time_str)
print(run_name)

time0 = time.time()

XGBoost_20181015-224246


## Import params

In [2]:
test_rows = 2

print(test_rows)

2


## Import pkgs

In [3]:
import os
import random
import time
import re
import multiprocessing
import json
from tqdm import tqdm_notebook

import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import seaborn
%matplotlib inline

from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, fbeta_score, jaccard_similarity_score

cpu_amount = multiprocessing.cpu_count()
random_num = np.random.randint(0, 9999)
random_num_str = '%04d' % random_num

print('cpu_amount: %s' % (cpu_amount - 1))
print('random_num: %s' % random_num_str)

C:\Users\study\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


cpu_amount: 3
random_num: 7778


## Project folders

In [4]:
cwd = os.getcwd()
feature_folder = os.path.join(cwd, 'feature')
input_folder = os.path.join(cwd, 'input')
output_folder = os.path.join(cwd, 'output')
model_folder = os.path.join(cwd, 'model')
log_folder = os.path.join(cwd, 'log')

print(feature_folder)
print(input_folder)
print(output_folder)
print(model_folder)
print(log_folder)

train_csv = os.path.join(input_folder, 'train.csv')
test_csv = os.path.join(input_folder, 'test.csv')
sample_submission_csv = os.path.join(input_folder, 'sample_submission.csv')
print(train_csv)
print(test_csv)
print(sample_submission_csv)

D:\Kaggle\ga-customer-revenue-prediction\feature
D:\Kaggle\ga-customer-revenue-prediction\input
D:\Kaggle\ga-customer-revenue-prediction\output
D:\Kaggle\ga-customer-revenue-prediction\model
D:\Kaggle\ga-customer-revenue-prediction\log
D:\Kaggle\ga-customer-revenue-prediction\input\train.csv
D:\Kaggle\ga-customer-revenue-prediction\input\test.csv
D:\Kaggle\ga-customer-revenue-prediction\input\sample_submission.csv


## Function

In [5]:
# https://www.kaggle.com/julian3833/1-quick-start-read-csv-and-flatten-json-fields

def load_df(csv_path, nrows=None, JSON_COLUMNS=['device', 'geoNetwork', 'totals', 'trafficSource']):
    data_df = pd.read_csv(csv_path, nrows=nrows)
    for idx, row in data_df.iterrows():
        for col in JSON_COLUMNS:
            if isinstance(data_df.at[idx, col], str):
                json_str = data_df.at[idx, col]
            if isinstance(data_df.at[idx, col], list):
                json_str = data_df.at[idx, col][0]
                print(len(data_df.at[idx, col]))
            json_obj = json.loads(json_str)
#             for key in json_obj:
#                 sub_col = '%s.%s' % (col, key)
#                 data_df.at[idx, sub_col] = json_obj[key]
#                 if sub_col == 'trafficSource.adwordsClickInfo':
#                     sub_json_str = data_df.at[idx, sub_col]
#                     sub_json_obj = json.loads(json_str)
#                     for sub_key in sub_json_obj:
#                         sub_sub_col = '%s.%s' % (sub_col, sub_key)
#                         data_df.at[idx, sub_sub_col] = json_obj[sub_key]
    data_df.drop(columns=['device', 'geoNetwork', 'totals', 'trafficSource'], inplace=True)
    return data_df

train_df = load_df(train_csv, test_rows)
display(train_df.shape, train_df.head(2))

(2, 8)

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime
0,Organic Search,20160902,1131660440785968503,1131660440785968503_1472830385,Not Socially Engaged,1472830385,1,1472830385
1,Organic Search,20160902,377306020877927890,377306020877927890_1472880147,Not Socially Engaged,1472880147,1,1472880147


In [31]:
def check_json_format(raw_msg): 
    """
    用于判断一个字符串是否符合Json格式
    """ 
    if isinstance(raw_msg, str): # 首先判断变量是否为字符串 
        try: 
            json.loads(raw_msg, encoding='utf-8') 
        except ValueError: 
            return False 
        return True 
    else: 
        return False


In [32]:
# https://www.kaggle.com/julian3833/1-quick-start-read-csv-and-flatten-json-fields

def load_df(csv_path, nrows=None, JSON_COLUMNS=['device', 'geoNetwork', 'totals', 'trafficSource']):
    df = pd.read_csv(csv_path, nrows=nrows)
    for idx, row in df.iterrows():
        for col in JSON_COLUMNS:
#             print(df.at[idx, col])
            json_str = df.at[idx, col]
            json_obj = json.loads(json_str)
            # Flatten json object to flattened key-value list
            key_prefix = [col]
            key_stack = [[json_obj, key, key_prefix] for key in json_obj]
            while(len(key_stack) > 0):
                item = key_stack.pop()
                obj = item[0]
                key = item[1]
                key_prefix = item[2]
                print(len(key_prefix), end=' ')
#                 print(key)
                if isinstance(obj[key], dict):
                    key_prefix.append(key)
                    print(len(key_prefix), end=' ')
                    sub_obj = obj[key]
                    key_stack = key_stack + [[sub_obj, sub_key, key_prefix] for sub_key in sub_obj]
                elif check_json_format(obj[key]):
                    key_prefix.append(key)
                    print(len(key_prefix), end=' ')
                    sub_obj = json.loads(obj[key])
                    if isinstance(sub_obj, dict):
                        key_stack = key_stack + [[sub_obj, sub_key, key_prefix] for sub_key in sub_obj]
                    elif isinstance(sub_obj, list):
                        for i in len(sub_obj):
                            key_stack = key_stack + [[sub_obj, sub_key, key_prefix] for sub_key in sub_obj]
                else:
                    if len(key_prefix) > 0:
                        key_str = '.'.join(key_prefix) + '.' + key
                    else:
                        key_str = key
                    if key_str not in df.columns:
                        df[key_str] = ''
                    print(key_str)
                    df.at[idx, key_str] = obj

    df.drop(columns=['device', 'geoNetwork', 'totals', 'trafficSource'], inplace=True)
    return df

train_df = load_df(train_csv, test_rows)
display(train_df.shape, train_df.head(2))

1 device.deviceCategory
1 device.screenResolution
1 device.screenColors
1 device.language
1 device.flashVersion
1 device.mobileDeviceMarketingName
1 device.mobileDeviceInfo
1 device.mobileInputSelector
1 device.mobileDeviceModel
1 device.mobileDeviceBranding
1 device.isMobile
1 device.operatingSystemVersion
1 device.operatingSystem
1 device.browserSize
1 device.browserVersion
1 device.browser
1 geoNetwork.networkLocation
1 geoNetwork.longitude
1 geoNetwork.latitude
1 geoNetwork.networkDomain
1 geoNetwork.cityId
1 geoNetwork.city
1 geoNetwork.metro
1 geoNetwork.region
1 geoNetwork.country
1 geoNetwork.subContinent
1 geoNetwork.continent
1 2 2 

TypeError: string indices must be integers

## Load and prevew data

In [ ]:
train_df = pd.read_csv(train_csv, nrows=test_rows)
test_df = pd.read_csv(test_csv, nrows=test_rows)
sample_submission_df = pd.read_csv(sample_submission_csv, nrows=test_rows)

display(train_df.columns)
display(train_df.shape, train_df.head(2))
display(test_df.shape, test_df.head(2))
display(sample_submission_df.shape, sample_submission_df.head(2))

## Load and preview data

## Decompose json object

In [ ]:

for idx, row in train_df.iterrows():
    for col in JSON_COLUMNS:
        if isinstance(train_df.at[idx, col], str):
            json_str = train_df.at[idx, col]
        else:
            json_str = train_df.at[idx, col][0]
        json_obj = json.loads(json_str)
        for key in json_obj:
            sub_col = '%s.%s' % (col, key)
            print(sub_col)
            train_df.at[idx, sub_col] = json_obj[key]

In [ ]:
print(run_name)
print('Time elapsed: %.2f s' % (time.time() - time0))